# OS Fingerprinting based on ML and nmap dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
seed = 2022
np.random.seed(seed)

#### Import dataset

In [3]:
df = pd.read_csv('./dataset.csv')

/tmp/ipykernel_39149/3350788483.py:1: DtypeWarning: Columns (7,9,10,30,39,41,45,46,47,48,50,52,56,57,58,59,61,63,67,68,69,70,72,81,83,85,89,90,91,92,94,96,100,101,102,103,105,106,115,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset.csv')


### Explore dataset

In [4]:
# print("Initial dataset")
# df.head()

In [5]:
# define output variable name
# OutVar = list(df.columns)[1]

In [6]:
# print(list(df.columns))

In [7]:
# def DataCheckings(df):
#     # Check the number of data points in the data set
#     print("\nData points =", len(df))
    
#     # Check the number of columns in the data set
#     print("\nColumns (output + features)=",len(df.columns))
    
#     # Check the data types
#     print("\nData types =", df.dtypes.unique())
    
#     # List of values per column
#     print()
#     for column in df.columns:
#         print(column + " -> ")
#         print(df[column].value_counts())
#         print()
    
#     # Dataset statistics
#     print('\n')
#     df.describe()
    
#     # print names of columns
#     print('Column Names:\n', df.columns)
    
#     # see if there are categorical data
#     print("\nCategorical features:", df.select_dtypes(include=['O']).columns.tolist())
    
#     # Check NA values
#     # Check any number of columns with NaN
#     print("\nColumns with NaN: ", df.isnull().any().sum(), ' / ', len(df.columns))

#     # Check any number of data points with NaN
#     print("\nNumber of data points with NaN:", df.isnull().any(axis=1).sum(), ' / ', len(df))

In [8]:
#DataCheckings(df)

In [9]:
df = df[['Class.OSfamily',
         'T1.R', 'T1.DF', 'T1.TG','WIN.W1','OPS.O1',
         'T1.R', 'T1.DF', 'T1.TG','WIN.W2','OPS.O2',
         'T1.R', 'T1.DF', 'T1.TG','WIN.W3','OPS.O3',
         'T1.R', 'T1.DF', 'T1.TG','WIN.W4','OPS.O4',
         'T1.R', 'T1.DF', 'T1.TG','WIN.W5','OPS.O5',
         'T1.R', 'T1.DF', 'T1.TG','WIN.W6','OPS.O6',
         'T2.R', 'T2.DF', 'T2.TG', 'T2.W',  'T2.O',   
         'T3.R', 'T3.DF', 'T3.TG', 'T3.W',  'T3.O',   
         'T4.R', 'T4.DF', 'T4.TG', 'T4.W',  'T4.O',   
         'T5.R', 'T5.DF', 'T5.TG', 'T5.W',  'T5.O',   
         'T6.R', 'T6.DF', 'T6.TG', 'T6.W',  'T6.O',   
         'T7.R', 'T7.DF', 'T7.TG', 'T7.W',  'T7.O'
         ]]

df.columns = ['os',
         '1.R', '1.DF', '1.TG','1.W','1.O',
         '2.R', '2.DF', '2.TG','2.W','2.O',
         '3.R', '3.DF', '3.TG','3.W','3.O',
         '4.R', '4.DF', '4.TG','4.W','4.O',
         '5.R', '5.DF', '5.TG','5.W','5.O',
         '6.R', '6.DF', '6.TG','6.W','6.O',
         '7.R', '7.DF', '7.TG', '7.W',  '7.O',   
         '8.R', '8.DF', '8.TG', '8.W',  '8.O',   
         '9.R', '9.DF', '9.TG', '9.W',  '9.O',   
         '10.R', '10.DF', '10.TG', '10.W', '10.O',   
         '11.R', '11.DF', '11.TG', '11.W', '11.O',   
         '12.R', '12.DF', '12.TG', '12.W', '12.O'
         ]

In [10]:
df_list = df.to_dict('records')
del df

In [11]:
new_df_list = []

for row in df_list:
    for i in range(1,13):
        if row[str(i)+'.R'] == "Y":
            new_row = {}
            istr = str(i)
            new_row['os'] = row['os']
            new_row['initial_ttl'] = row[istr+'.TG']
            new_row['window_size'] = row[istr+'.W']
            new_row['quirk_df'] = 1 if row[istr+'.DF'] == "Y" else 0
            new_row['quirk_ts'] = 0
            new_row['mss'] = '*'
            new_row['window_scaling'] = '*'
            
            options = str(row[istr+'.O'])
            options = options.split(',')
            final_options = []
            for options_item in options:
                if options_item in ["eol","nop","sok"]:
                    final_options.append(options_item)
                if "mss" in options_item :
                    new_row['mss'] = options_item.split('.')[1]
                    final_options.append("mss")
                if "ws" in options_item:
                    new_row['window_scaling'] = options_item.split('.')[1]
                    final_options.append("ws")
                if "ts" in options_item:
                    new_row['quirk_ts'] = 1 if options_item.split('.')[1][0] == '0' else 0
                    final_options.append("ts")
                    
            new_row['tcp_options'] = ','.join(final_options)
            new_df_list.append(new_row)
            
del df_list

In [12]:
df = pd.DataFrame(new_df_list)
del new_df_list

In [13]:
print('Shape before removing duplicates = ', df.shape)

Shape before removing duplicates =  (2689933, 8)


In [14]:
df.drop_duplicates(keep=False, inplace=True)
print('Shape after removing duplicates=', df.shape)

Shape after removing duplicates= (1105, 8)


### Encoding of the dataset

In [15]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from ai_model_creation.ai_nmap_model_creation.transformers import *

#### Filter Operating Systems

In [16]:
# Filter OSes
# df = df[df.os.isin(['Linux', 'Windows', 'Mac OS X', 'Solaris', 'OpenBSD', 'FreeBSD'])]

df.reset_index(inplace=True, drop=True)

#### Data Augmentation

In [17]:
# Data Augmentation
# TTL, MSS & Windows size

# ttl_factor = 10

# array = df.to_numpy()
# ttl_i = df.columns.get_loc('initial_ttl')

# for row in array:
#     for i in range(1,ttl_factor+1):
#         new_row = row.copy()
#         new_row[ttl_i] = row[ttl_i] - i
        
#         array = np.vstack((array, new_row))
        
# df = pd.DataFrame(array, columns = df.columns)

#### TTL

In [18]:
# TTL
# Numeric value (¿Standarization, Normalization?)

#### MSS

In [19]:
# MSS
# Categorical encoding

encoder_mss = OneHotEncoder(drop=['*'], sparse=False, handle_unknown='ignore')

#### Window Size

In [20]:
# Window Size
# Drop column

encoder_window_size = WindowSizeTransformer()

#### Windows Scaling

In [21]:
# Window Scaling 
# Categorical encoding

encoder_window_scaling = OneHotEncoder(drop=['*'], sparse=False, handle_unknown='ignore')

# encoder_window_scaling.fit(df[['window_scaling']])

#### TCP options

In [22]:
# TCP Options
# Custom transformer

encoder_tcp_options = TCPOptionsTransformer()

#### Quirks

In [23]:
# Quirks
# Categorical encoding (already encoded)

#### Applying encodings

In [24]:
# Apply encodings

from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

df.reset_index(inplace=True, drop=True)

encoders = make_column_transformer(
    (encoder_mss, ['mss']),
    (encoder_window_size, ['window_size']),
    (encoder_window_scaling, ['window_scaling']),
    (encoder_tcp_options, ['tcp_options']),
    remainder='passthrough',
    verbose_feature_names_out=False)

transformed = encoders.fit_transform(df)
transformed_df = pd.DataFrame(
    transformed,
    columns=encoders.get_feature_names_out()
)

df = transformed_df

# define output variable name
OutVar = df.os.name

In [25]:
# Remove duplicates

df = df.drop_duplicates()

In [26]:
df[OutVar].value_counts()

Linux      365
Windows    308
macOS       44
iOS         43
Solaris     42
FreeBSD     35
Android     20
OpenBSD     19
NetBSD       9
Name: os, dtype: int64

In [27]:
# Request

Ydata = df[OutVar].values                  # get values of features
Xdata = df.drop(OutVar,axis = 1).values    # get output values

print('Shape X data:', Xdata.shape)
print('Shape Y data:',Ydata.shape)

Shape X data: (885, 249)
Shape Y data: (885,)


### Data split

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
# Request

X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata,
                                                    stratify=Ydata, 
                                                    test_size=0.10,
                                                    random_state=seed)

# verify dimentions of data for training and test
print('Shape X_train:', X_train.shape)
print('Shape X_test:' , X_test.shape)
print('Shape y_train:', y_train.shape)
print('Shape y_test:' , y_test.shape)

Shape X_train: (796, 249)
Shape X_test: (89, 249)
Shape y_train: (796,)
Shape y_test: (89,)


### ML

In [30]:
import time
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score, roc_auc_score,f1_score, recall_score, precision_score
from sklearn.utils import class_weight
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

##### Classes balance

In [31]:
def set_weights(y_data, option='balanced'):
    """Estimate class weights for umbalanced dataset
       If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)). 
       If a dictionary is given, keys are classes and values are corresponding class weights. 
       If None is given, the class weights will be uniform """
    cw = class_weight.compute_class_weight(class_weight=option, classes=np.unique(y_data), y=y_data)
    w = {i:j for i,j in zip(np.unique(y_data), cw)}
    return w

class_weights = set_weights(Ydata)

print("Request balance => ",class_weights)

Request balance =>  {'Android': 4.916666666666667, 'FreeBSD': 2.8095238095238093, 'Linux': 0.2694063926940639, 'NetBSD': 10.925925925925926, 'OpenBSD': 5.175438596491228, 'Solaris': 2.3412698412698414, 'Windows': 0.31926406926406925, 'iOS': 2.2868217054263567, 'macOS': 2.234848484848485}


##### Classifiers definition

In [32]:
# define a list of classifiers to train as baseline classifiers
classifiers = [
    GaussianNB(),
    LinearDiscriminantAnalysis(), # No random_state
    LogisticRegression(n_jobs=-1,solver='lbfgs',random_state=seed,class_weight=class_weights),
    MLPClassifier(hidden_layer_sizes= (30), random_state = seed, shuffle=False, solver='adam',activation='relu',batch_size=500, max_iter=5000),
    DecisionTreeClassifier(random_state=seed,class_weight=class_weights),
    RandomForestClassifier(n_jobs=-1,random_state=seed,class_weight=class_weights),
    BaggingClassifier(n_jobs=-1,random_state=seed)
]

##### Training

In [33]:
# training and metrics (ACC, precision, recall, f1score) for a classifier
def ML_baseline(cls, X_tr, y_tr, X_ts, y_ts, seed=42, classes=['0','1']):
    ACC = 0
    AUROC = 0
    precision = 0 
    recall = 0
    f1score = 0
    
    cls_name = type(cls).__name__
    
    start_time = time.time()
    cls.fit(X_tr, y_tr) # TRAINING!
    # print('\n---->', "training: %0.2f mins \n\n" % ((time.time() - start_time)/60))
    
    # predictions
    y_pred  = cls.predict(X_ts)             # predict classes
    y_probs = cls.predict_proba(X_ts)[:, 1] # predict probabilities of classes
    cls_rep = classification_report(y_ts, y_pred, target_names=classes,
                                    output_dict=True, digits=3)
    # print classification report
    #print(cls_rep)
    
    ACC       = accuracy_score(y_ts, y_pred)
    #AUROC     = roc_auc_score(y_ts, y_probs) # this is working for 2-classes classification only!!!
    precision = cls_rep['weighted avg']['precision']
    recall    = cls_rep['weighted avg']['recall']
    f1score   = cls_rep['weighted avg']['f1-score']  
    
    # print metrics
    # print("\n", "ACC=", ACC, "precision=", precision, "recall=", recall, "f1score=",f1score)
    
    return cls, ACC, precision, recall, f1score

In [34]:
# Response

# create a dataframe for ML baseline
statistics_ML = pd.DataFrame(columns=['Method', 'ACC','precision' ,'recall' ,'f1-score' ])
models_ML = pd.DataFrame(columns=['Method', 'Model' ])

classes_names = np.unique(Ydata)

for cls in classifiers:
    cls_fit, ACC, precision,recall,f1score=ML_baseline(cls, X_train, y_train, X_test, y_test, seed=seed,classes=classes_names)
    
    statistics_ML = statistics_ML.append({'Method': str(type(cls).__name__),
                                                            'ACC': float(ACC),
                                                            #'AUROC': float(AUROC),
                                                            'precision': float(precision),
                                                            'recall': float(recall),
                                                            'f1-score': float(f1score)}, ignore_index=True)
    
    models_ML = models_ML.append({'Method': str(type(cls).__name__),
                                                    'Model' : cls_fit}, ignore_index=True)

/tmp/ipykernel_39149/3929213877.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  statistics_ML = statistics_ML.append({'Method': str(type(cls).__name__),
/tmp/ipykernel_39149/3929213877.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_ML = models_ML.append({'Method': str(type(cls).__name__),
/home/ruben/AIFingerprintingTool/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ruben/AIFingerprintingTool/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and bei

In [35]:
statistics_ML

,Method,ACC,precision,recall,f1-score
0,GaussianNB,0.662921,0.817769,0.662921,0.709844
1,LinearDiscriminantAnalysis,0.775281,0.755377,0.775281,0.758809
2,LogisticRegression,0.730337,0.812306,0.730337,0.756722
3,MLPClassifier,0.808989,0.774836,0.808989,0.788025
4,DecisionTreeClassifier,0.741573,0.759765,0.741573,0.743567
5,RandomForestClassifier,0.786517,0.786454,0.786517,0.781056
6,BaggingClassifier,0.808989,0.828777,0.808989,0.806547


### Export transformers & model

In [36]:
from joblib import dump, load

# Transformers
dump(encoders, '../../persistence/nmap/nmap_encoders.joblib')

# Models
dump(models_ML.Model.values[6],'../../persistence/nmap/nmap_classifier.joblib')

['../../persistence/nmap_classifier.joblib']